# 2019/09/21 ver.1 模板
## 一、文件内容
特征选择：GBRT <br />
单模型构建：<br />
 1：SVR<br />
 2：RF<br />
 3：XGBoost<br />
 4：KNN<br />
 5：GBRT<br />
Stacking：<br />
 SVR默认
## 二、数据集
### 导入数据集变量<br />
元数据集 训练集全部：dg_train →训练集 特征数据：dg_train<br />
元数据集 测试集全部：dg_test  →测试集 特征数据：dg_test<br />
<br />
训练集 得分数据：score_train<br />
测试集 序号：id_test<br />
训练集 序号：id_train
<br />
### 处理数据集变量<br />
标准化处理后的原训练集 特征数据：dg_scaled_train<br />
标准化处理后的测试集合 特征数据：dg_scaled_test<br />
### 特征选择结果
特征选择后的训练集 特征数据：X<br />
特征选择后的测试集 特征数据：X_predict<br />
### 在X中继续划分数据<br />
X_train：0.8比例的原训练集特征 用作训练和验证<br />
X_test：0.2比例的原训练集特征 用作测试（模型选择）<br />
y_train：0.8比例的原训练集分数 对应X_train<br />
y_test：0.2比例的原训练集分数 对应X_test<br />

## 三、模型信息


变量名|模型名|超参数设置|随机数种子|训练集上预测值
:---------:|:------------------:|:-----------------:|:-----------------:|:-------------------------:
model_svc|支持向量回归||NA|
model_svcRF|随机森林回归||160|
model_xgbr|Xgboost回归||161|
model_k_neighbor|KNN回归||NA|
model_gradient_boosting_regressor|GBRT||162|

## 四、随机数信息
位置|使用函数|随机数种子
:------:|:-------:|:--------:|
特征处理|GBRT|20
特征处理|train_test_split|21
stacking|KFold|22
预测|train_test_split|23


# 1、前処理（preprocess）

In [1]:
#预设导入
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import joblib

import warnings
warnings.filterwarnings("ignore")

In [2]:
#机器学习导入
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import explained_variance_score 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import r2_score
from sklearn import ensemble
from sklearn import neighbors
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

In [3]:
dg_train=pd.read_csv('train.csv')
dg_test=pd.read_csv("test.csv")

#单独提取ID列和score列
id_train=dg_train["ID"].values
id_test=dg_test["ID"].values
score_train=dg_train["Score"].values

del dg_train["ID"]
del dg_train["Score"]
del dg_test["ID"]

In [4]:
#z-score 标准化
scaler=StandardScaler().fit(dg_train)#标准化的mean var
print(scaler)
print(scaler.mean_)
print(scaler.var_)
dg_scaled_train=scaler.transform(dg_train)#标准化结果向量
print(dg_scaled_train)
np.shape(dg_scaled_train)

dg_scaled_test=scaler.transform(dg_test)
print(dg_scaled_test)
np.shape(dg_scaled_test)#相同标准 标准化测试集

StandardScaler(copy=True, with_mean=True, with_std=True)
[5.82623261e-04 4.17374998e+00 7.18538490e-01 ... 1.87939626e+00
 9.97324521e-01 1.03583133e+00]
[7.27939627e-04 7.21597472e-02 1.39518638e-02 ... 2.31160294e+00
 6.76420780e-07 3.08326932e+00]
[[-0.02159435 -0.43089696  0.8928487  ... -0.96316788 -0.39457952
  -0.58990668]
 [-0.02159435 -0.23359638  1.78179021 ...  1.38095879 -0.39457952
   1.11859545]
 [-0.02159435 -0.12936211  0.96904369 ... -0.67771587  2.03718504
  -0.58990668]
 ...
 [-0.02159435  1.25174197  0.83358594 ...  0.55025397  0.82130276
  -0.58990668]
 [-0.02159435  1.17728892 -0.25007609 ... -1.39923858  0.82130276
  -0.58990668]
 [-0.02159435 -0.48301409 -0.45326272 ...  0.18061334 -0.39457952
  -0.58990668]]
[[-0.02159435 -0.42717431 -0.74957655 ...  0.41476292  0.82130276
  -0.58990668]
 [-0.02159435 -1.50674353  0.78278928 ... -0.40936467 -0.39457952
  -0.58990668]
 [-0.02159435  1.90320616  0.7150604  ...  0.67851006  0.82130276
  -0.58990668]
 ...
 [-0.0215

(13732, 3805)

### 以下开始特征选择

In [5]:
#基模型的特征选择
clf_fs = ensemble.GradientBoostingRegressor(random_state=20)
clf_fs =clf_fs.fit(dg_scaled_train, score_train)
clf_fs.feature_importances_  

model_fs = SelectFromModel(clf_fs, prefit=True)
X = model_fs.transform(dg_scaled_train)


#SelectFromModel(ensemble.GradientBoostingRegressor()).fit_transform(dg_scaled_train, score_train)

In [6]:
X_predict=model_fs.transform(dg_scaled_test)
print(dg_scaled_test)
np.shape(X_predict)

[[-0.02159435 -0.42717431 -0.74957655 ...  0.41476292  0.82130276
  -0.58990668]
 [-0.02159435 -1.50674353  0.78278928 ... -0.40936467 -0.39457952
  -0.58990668]
 [-0.02159435  1.90320616  0.7150604  ...  0.67851006  0.82130276
  -0.58990668]
 ...
 [-0.02159435 -1.04513462  1.66326468 ... -1.17561258 -0.39457952
  -0.58990668]
 [-0.02159435  0.41042251 -0.65644935 ... -0.00815331 -0.39457952
  -0.58990668]
 [-0.02159435 -0.73987712  0.12243273 ... -0.52512401  0.82130276
  -0.58990668]]


(13732, 356)

In [7]:
np.shape(X)

(13731, 356)

In [8]:
#划分数据集

X_train, X_test, y_train, y_test = train_test_split(X, score_train, test_size=0.2, random_state=21)
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((10984, 356), (10984,), (2747, 356), (2747,))

# 2、学習（train）

### SVR

In [9]:
#SVR模型实现
model_svc=svm.SVR()
model_svc.fit(X_train,y_train)

print(model_svc.score(X_test,y_test))

y_pred = model_svc.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))

0.5894213065555851
0.5894213065555851


In [10]:
#SVR模型调参数：C gamma
param_g=[{"C": [1e0, 1e1, 1e2, 1e3],"gamma": np.logspace(-4, 0, 5)}]
scores_g=['r2']
for score in scores_g:
    print(score)
    clf=GridSearchCV(svm.SVR(),param_grid=param_g,cv=3,scoring=score)
    clf.fit(X_train,y_train)
    
    print(clf.best_estimator_)
    
    means = clf.cv_results_['mean_test_score']
    stds=clf.cv_results_['std_test_score']
    params = clf.cv_results_['params']
    
    for (param,mean_score,std_score) in zip(params,means,stds):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score,std_score,param))

r2
SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.001,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
0.381(+/-0.006) for {'C': 1.0, 'gamma': 0.0001}
0.499(+/-0.007) for {'C': 1.0, 'gamma': 0.001}
0.508(+/-0.006) for {'C': 1.0, 'gamma': 0.01}
0.045(+/-0.002) for {'C': 1.0, 'gamma': 0.1}
0.011(+/-0.000) for {'C': 1.0, 'gamma': 1.0}
0.439(+/-0.007) for {'C': 10.0, 'gamma': 0.0001}
0.535(+/-0.007) for {'C': 10.0, 'gamma': 0.001}
0.515(+/-0.009) for {'C': 10.0, 'gamma': 0.01}
0.052(+/-0.003) for {'C': 10.0, 'gamma': 0.1}
0.013(+/-0.001) for {'C': 10.0, 'gamma': 1.0}
0.488(+/-0.007) for {'C': 100.0, 'gamma': 0.0001}
0.470(+/-0.006) for {'C': 100.0, 'gamma': 0.001}
0.516(+/-0.009) for {'C': 100.0, 'gamma': 0.01}
0.052(+/-0.003) for {'C': 100.0, 'gamma': 0.1}
0.012(+/-0.002) for {'C': 100.0, 'gamma': 1.0}
0.490(+/-0.004) for {'C': 1000.0, 'gamma': 0.0001}
0.435(+/-0.006) for {'C': 1000.0, 'gamma': 0.001}
0.509(+/-0.017) for {'C': 1000.0, 'gamma

In [11]:
#SVR模型实现
model_svc_t=clf.best_estimator_
model_svc_t.fit(X_train,y_train)

print(model_svc_t.score(X_test,y_test))

y_pred = model_svc_t.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))

model_svc=clf.best_estimator_

0.5884181686444214
0.5884181686444214


### RF

In [12]:
model_svcRF=RandomForestRegressor(random_state=160)
model_svcRF.fit(X_train,y_train)

print(model_svcRF.score(X_test,y_test))#0.4697676007383277

y_pred = model_svcRF.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))

0.48585311378303303
0.48585311378303303


In [13]:
N_estimators=[{"n_estimators":[20,50,100,150,200,300,400,500]}]
scores_g=['r2']
for score in scores_g:
    print(score)
    clf=GridSearchCV(RandomForestRegressor(random_state=160),param_grid=N_estimators,cv=3,scoring=score)
    clf.fit(X_train,y_train)
    
    print(clf.best_estimator_)
    
    means = clf.cv_results_['mean_test_score']
    stds=clf.cv_results_['std_test_score']
    params = clf.cv_results_['params']
    
    for (param,mean_score,std_score) in zip(params,means,stds):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score,std_score,param))

r2
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=160, verbose=0,
                      warm_start=False)
0.484(+/-0.009) for {'n_estimators': 20}
0.501(+/-0.012) for {'n_estimators': 50}
0.507(+/-0.012) for {'n_estimators': 100}
0.509(+/-0.013) for {'n_estimators': 150}
0.510(+/-0.014) for {'n_estimators': 200}
0.511(+/-0.012) for {'n_estimators': 300}
0.512(+/-0.012) for {'n_estimators': 400}
0.513(+/-0.013) for {'n_estimators': 500}


In [14]:
model_svcRF_t=clf.best_estimator_
model_svcRF_t.fit(X_train,y_train)

print(model_svcRF_t.score(X_test,y_test))#默认0.4697676007383277

y_pred = model_svcRF_t.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))
model_svcRF=clf.best_estimator_

0.5486919128328054
0.5486919128328054


### xgb

In [15]:
model_xgbr = xgb.XGBRegressor(random_state=161)
model_xgbr.fit(X_train, y_train)

print(model_xgbr.score(X_test,y_test))

y_pred = model_xgbr.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))

[04:19:45] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.4851841419105461
0.4851841419105461


In [16]:
#调参 XGboost——n_estimators 其他默认
cv_params = [{"n_estimators":[20,50,100,150,200,300,400,500]}]

scores_GBM=['r2']

for score in scores_GBM:
    print(score)
    model = XGBRegressor(random_state=161)
    optimized_GBM=GridSearchCV(estimator=model, param_grid=cv_params, scoring=score, cv=3, verbose=1, n_jobs=-1)
    optimized_GBM.fit(X_train,y_train)
    
    print(optimized_GBM.best_estimator_)
    
    means_GBM = optimized_GBM.cv_results_['mean_test_score']
    stds_GBM=optimized_GBM.cv_results_['std_test_score']
    params_GBM = optimized_GBM.cv_results_['params']
    
    for (param_GBM,mean_score_GBM,std_score_GBM) in zip(params_GBM,means_GBM,stds_GBM):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score_GBM,std_score_GBM,param_GBM))

r2
Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  3.0min finished


[04:22:55] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=161,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)
0.295(+/-0.011) for {'n_estimators': 20}
0.422(+/-0.012) for {'n_estimators': 50}
0.472(+/-0.011) for {'n_estimators': 100}
0.495(+/-0.011) for {'n_estimators': 150}
0.508(+/-0.012) for {'n_estimators': 200}
0.524(+/-0.013) for {'n_estimators': 300}
0.533(+/-0.012) for {'n_estimators': 400}
0.538(+/-0.012) for {'n_estimators': 500}


In [17]:
#调参 XGboost——maxdepth minchildweight 其他默认
cv_params = [{'max_depth': [3, 5,7,9], 'min_child_weight': [1, 3, 5]}]

scores_GBM_mami=['r2']

for score in scores_GBM_mami:
    print(score)
    model = optimized_GBM.best_estimator_
    optimized_GBM_mami=GridSearchCV(estimator=model, param_grid=cv_params, scoring=score, cv=3, verbose=1, n_jobs=-1)
    optimized_GBM_mami.fit(X_train,y_train)
    
    print(optimized_GBM_mami.best_estimator_)
    
    means_GBM_mami = optimized_GBM_mami.cv_results_['mean_test_score']
    stds_GBM_mami=optimized_GBM_mami.cv_results_['std_test_score']
    params_GBM_mami = optimized_GBM_mami.cv_results_['params']
    
    for (param_GBM_mami,mean_score_GBM_mami,std_score_GBM_mami) in zip(params_GBM_mami,means_GBM_mami,stds_GBM_mami):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score_GBM_mami,std_score_GBM_mami,param_GBM_mami))

r2
Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 21.0min finished


[04:44:50] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=161,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)
0.538(+/-0.012) for {'max_depth': 3, 'min_child_weight': 1}
0.537(+/-0.009) for {'max_depth': 3, 'min_child_weight': 3}
0.537(+/-0.011) for {'max_depth': 3, 'min_child_weight': 5}
0.558(+/-0.007) for {'max_depth': 5, 'min_child_weight': 1}
0.559(+/-0.012) for {'max_depth': 5, 'min_child_weight': 3}
0.560(+/-0.011) for {'max_depth': 5, 'min_child_weight': 5}
0.561(

In [18]:
#调参 XGboost——学习率
cv_params = [{'learning_rate':  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]}]

scores_GBM_lr=['r2']

for score in scores_GBM_lr:
    print(score)
    model = optimized_GBM_mami.best_estimator_
    optimized_GBM_lr=GridSearchCV(estimator=model, param_grid=cv_params, scoring=score, cv=3, verbose=1, n_jobs=-1)
    optimized_GBM_lr.fit(X_train,y_train)
    
    print(optimized_GBM_lr.best_estimator_)
    
    means_GBM_lr = optimized_GBM_lr.cv_results_['mean_test_score']
    stds_GBM_lr=optimized_GBM_lr.cv_results_['std_test_score']
    params_GBM_lr = optimized_GBM_lr.cv_results_['params']
    
    for (param_GBM_lr,mean_score_GBM_lr,std_score_GBM_lr) in zip(params_GBM_lr,means_GBM_lr,stds_GBM_lr):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score_GBM_lr,std_score_GBM_lr,param_GBM_lr))

r2
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 13.3min finished


[05:00:28] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=161,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)
-2.447(+/-0.011) for {'learning_rate': 0.0001}
-0.769(+/-0.010) for {'learning_rate': 0.001}
0.532(+/-0.011) for {'learning_rate': 0.01}
0.561(+/-0.010) for {'learning_rate': 0.1}
0.539(+/-0.014) for {'learning_rate': 0.2}
0.507(+/-0.011) for {'learning_rate': 0.3}


In [19]:
model_xgbr_t = optimized_GBM.best_estimator_
model_xgbr_t.fit(X_train, y_train)

print(model_xgbr_t.score(X_test,y_test))

y_pred = model_xgbr_t.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))
model_xgbr = optimized_GBM.best_estimator_

[05:02:49] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.5671934948663737
0.5671934948663737


### KNN

In [20]:
# 4.kNN回归
model_k_neighbor = neighbors.KNeighborsRegressor()
model_k_neighbor.fit(X_train, y_train)

print(model_k_neighbor.score(X_test,y_test))

y_pred = model_k_neighbor.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))

0.4610388756489285
0.4610388756489285


In [21]:
param_grids=[
    {
        'weights':['uniform'],
        'n_neighbors':[i for i in range(1,11)]
    },
    {
        'weights':['distance'],
        'n_neighbors':[i for i in range(1,11)],
        'p':[i for i in range(1,6)]
    }]
scores_KNN=['r2']

for score in scores_KNN:
    print(score)
    model = neighbors.KNeighborsRegressor()
    clf_k=GridSearchCV(estimator=model, param_grid=param_grids, scoring=score, cv=3, verbose=1, n_jobs=-1)
    clf_k.fit(X_train,y_train)
    
    print(clf_k.best_estimator_)
    
    means_KNN = clf_k.cv_results_['mean_test_score']
    stds_KNN=clf_k.cv_results_['std_test_score']
    params_KNN = clf_k.cv_results_['params']
    
    for (param_KNN,mean_score_KNN,std_score_KNN) in zip(params_KNN,means_KNN,stds_KNN):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score_KNN,std_score_KNN,param_KNN))

r2
Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed: 133.2min finished


KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=8, p=1,
                    weights='distance')
0.192(+/-0.012) for {'n_neighbors': 1, 'weights': 'uniform'}
0.347(+/-0.007) for {'n_neighbors': 2, 'weights': 'uniform'}
0.391(+/-0.006) for {'n_neighbors': 3, 'weights': 'uniform'}
0.411(+/-0.007) for {'n_neighbors': 4, 'weights': 'uniform'}
0.417(+/-0.010) for {'n_neighbors': 5, 'weights': 'uniform'}
0.416(+/-0.009) for {'n_neighbors': 6, 'weights': 'uniform'}
0.416(+/-0.011) for {'n_neighbors': 7, 'weights': 'uniform'}
0.414(+/-0.011) for {'n_neighbors': 8, 'weights': 'uniform'}
0.412(+/-0.013) for {'n_neighbors': 9, 'weights': 'uniform'}
0.413(+/-0.012) for {'n_neighbors': 10, 'weights': 'uniform'}
0.220(+/-0.011) for {'n_neighbors': 1, 'p': 1, 'weights': 'distance'}
0.192(+/-0.012) for {'n_neighbors': 1, 'p': 2, 'weights': 'distance'}
0.154(+/-0.021) for {'n_neighbors': 1, 'p': 3, 'weights': 'dista

In [22]:
model_k_neighbor_t = clf_k.best_estimator_
model_k_neighbor_t.fit(X_train, y_train)

print(model_k_neighbor_t.score(X_test,y_test))

y_pred = model_k_neighbor_t.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))

model_k_neighbor = clf_k.best_estimator_

0.48903204767072717
0.48903204767072717


### GBRT

In [23]:
model_gradient_boosting_regressor = ensemble.GradientBoostingRegressor(random_state=162)  

model_gradient_boosting_regressor.fit(X_train, y_train)

print(model_gradient_boosting_regressor.score(X_test,y_test))

y_pred = model_gradient_boosting_regressor.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))

0.4878077429565811
0.4878077429565811


In [24]:
N_estimators=[{"n_estimators":[20,50,100,150,200,300,400,500]}]
scores_g=['r2']
for score in scores_g:
    print(score)
    clf_gb=GridSearchCV(ensemble.GradientBoostingRegressor(random_state=162),param_grid=N_estimators,cv=3,scoring=score)
    clf_gb.fit(X_train,y_train)
    
    print(clf_gb.best_estimator_)
    
    means = clf_gb.cv_results_['mean_test_score']
    stds=clf_gb.cv_results_['std_test_score']
    params = clf_gb.cv_results_['params']
    
    for (param,mean_score,std_score) in zip(params,means,stds):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score,std_score,param))

r2
GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=500,
                          n_iter_no_change=None, presort='auto',
                          random_state=162, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)
0.335(+/-0.011) for {'n_estimators': 20}
0.423(+/-0.012) for {'n_estimators': 50}
0.472(+/-0.010) for {'n_estimators': 100}
0.495(+/-0.011) for {'n_estimators': 150}
0.507(+/-0.010) for {'n_estimators': 200}
0.522(+/-0.009) for {'n_estimators': 300}
0.530(+/-0.010) for {'n_estimators': 400}
0.535(+/-0.010) for {'n_estimators': 500}


In [25]:
model_gradient_boosting_regressor_t = clf_gb.best_estimator_

model_gradient_boosting_regressor_t.fit(X_train, y_train)

print(model_gradient_boosting_regressor_t.score(X_test,y_test))

y_pred = model_gradient_boosting_regressor_t.predict(X_test)
print(r2_score(y_test,y_pred, multioutput='variance_weighted'))

model_gradient_boosting_regressor = clf_gb.best_estimator_

0.5685898225956898
0.5685898225956898


# 第二阶段stacking

In [26]:
# '''创建训练的数据集'''
#data, target = make_blobs(n_samples=50000, centers=2, random_state=0, cluster_std=0.60)
 
# '''模型融合中使用到的各个单模型'''
clfs = [model_svc,
        model_svcRF,
        model_xgbr,
        model_k_neighbor,
        model_gradient_boosting_regressor
       ]
 
#'''切分一部分数据作为测试集'''
X=X#训练集 数据
X_predict=X_predict#测试集 数据
y=score_train#训练集 分数
#y_predict = 


dataset_blend_train = np.zeros((X.shape[0], len(clfs)))#第一轮 保存各个模型在训练集上的预测结果 训练集合个数×模型数
dataset_blend_test = np.zeros((X_predict.shape[0], len(clfs)))#第一轮 保存各个模型在测试集上的预测结果 训练集合个数×模型数

#'''5折stacking'''
n_folds = 5
kf = KFold(n_folds,True,22)
skf=list(kf.split(X))#X或者y

for j, clf in enumerate(clfs):
    #'''依次训练各个单模型'''
    print(j, clf)
    dataset_blend_test_j = np.zeros((X_predict.shape[0], len(skf)))#存目前这个模型上的测试集结果(之后求平均)
    for i, (train, test) in enumerate(skf):
        #'''使用第i个部分作为预测，剩余的部分来训练模型，获得其预测的输出作为第i部分的新特征。'''
        print("Fold", i)
        X_train, y_train, X_test, y_test = X[train], y[train], X[test], y[test]
        clf.fit(X_train, y_train)
        y_submission = clf.predict(X_test)#1fold的预测结果
        y_submission=y_submission.flatten()#2维数组转1维 KNN需要 
        #y_submission.reshape(len(y_submission),1)#一维数组转二维 可以不加
        
        dataset_blend_train[test, j] = y_submission#在模型顺序对应的j位置 存1fold的预测结果
        dataset_blend_test_j[:, i] = clf.predict(X_predict).flatten()#存该模型该折下的测试集预测结果
        
    #'''对于测试集，直接用这k个模型的预测值均值作为新的特征。'''
    dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)#测试集结果按行取平均后储存
    
    
    #print("val auc Score: %f" % r2_score(y_predict, dataset_blend_test[:, j]))
    

0 SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.001,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
1 RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=160, verbose=0,
                      warm_start=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
2 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=

In [27]:
#导出数据集到本地
submission_train_1=pd.DataFrame(dataset_blend_train)
#submission_train_1.head()
submission_train_1.to_csv('dataset_blend_train.csv',index=False)#第一轮训练后 train集合预测得到的score集合 训练集样本数x3个模型

submission_test_1=pd.DataFrame(dataset_blend_test)
#submission_test_1.head()
submission_test_1.to_csv('dataset_blend_test.csv',index=False)#第一轮训练后 test集合预测得到的score集合 测试机样本数x3个模型

# 3、予測（predict）

In [28]:
#第二轮 
model_stacking_svr = svm.SVR()

model_stacking_svr.fit(dataset_blend_train, y)
y_submission = model_stacking_svr.predict(dataset_blend_test)

#生成文件
submission_df=pd.DataFrame(data={'Id':id_test,'SalePrice':y_submission})
submission_df.head()
submission_df.to_csv('baseline_st2_test1.csv',header=False,index=False)

In [29]:
#调参数
X_train, X_test, y_train, y_test = train_test_split(dataset_blend_train, y, test_size=0.2, random_state=123)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

#SVR模型调参数：C gamma
param_g=[{"C": [1e0, 1e1, 1e2, 1e3],"gamma": np.logspace(-4, 0, 5)}]
scores_g=['r2']
for score in scores_g:
    print(score)
    clf_st=GridSearchCV(svm.SVR(),param_grid=param_g,cv=3,scoring=score)
    clf_st.fit(X_train,y_train)
    
    print(clf_st.best_estimator_)
    
    means = clf_st.cv_results_['mean_test_score']
    stds=clf_st.cv_results_['std_test_score']
    params = clf_st.cv_results_['params']
    
    for (param,mean_score,std_score) in zip(params,means,stds):
        print("%0.3f(+/-%0.03f) for %r"%(mean_score,std_score,param))

(10984, 5) (10984,) (2747, 5) (2747,)
r2
SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
0.594(+/-0.005) for {'C': 1.0, 'gamma': 0.0001}
0.614(+/-0.006) for {'C': 1.0, 'gamma': 0.001}
0.615(+/-0.007) for {'C': 1.0, 'gamma': 0.01}
0.616(+/-0.007) for {'C': 1.0, 'gamma': 0.1}
0.613(+/-0.007) for {'C': 1.0, 'gamma': 1.0}
0.614(+/-0.006) for {'C': 10.0, 'gamma': 0.0001}
0.615(+/-0.007) for {'C': 10.0, 'gamma': 0.001}
0.615(+/-0.007) for {'C': 10.0, 'gamma': 0.01}
0.617(+/-0.007) for {'C': 10.0, 'gamma': 0.1}
0.607(+/-0.005) for {'C': 10.0, 'gamma': 1.0}
0.615(+/-0.007) for {'C': 100.0, 'gamma': 0.0001}
0.615(+/-0.007) for {'C': 100.0, 'gamma': 0.001}
0.615(+/-0.007) for {'C': 100.0, 'gamma': 0.01}
0.616(+/-0.007) for {'C': 100.0, 'gamma': 0.1}
0.576(+/-0.009) for {'C': 100.0, 'gamma': 1.0}
0.614(+/-0.007) for {'C': 1000.0, 'gamma': 0.0001}
0.615(+/-0.007) for {'C': 1000.0, 'gamma': 0.001}
0.61

In [31]:
clf_st_best = clf_st.best_estimator_

clf_st_best.fit(dataset_blend_train, y)
y_submission = clf_st_best.predict(dataset_blend_test)

#生成文件
submission_df=pd.DataFrame(data={'Id':id_test,'SalePrice':y_submission})
submission_df.head()
submission_df.to_csv('baseline_st2_test2.csv',header=False,index=False)